In [ ]:
import sys,os
sys.path.append('..')
from backtester import matlab, backtester
from backtester.analysis import *
from backtester.strategy import StrategyBase, OptParam
from backtester.swarms import SwarmManager, SwarmRanker

import pandas as pd
import numpy as np
import scipy

In [ ]:
%pylab inline
%load_ext autoreload
%autoreload 2

## Script settings

In [ ]:
# File name of exo strategy
EXO_File = 'strategy_270225'


#
# Costs settings
#
costs_options = 3.0 # 3$ per side per option contract
costs_futures = 3.0 # 3$ per side per futures contract



#
# Swarm settings
#

# number of strategies to pick  from the swarm
NSwarm_Members = 5

#
# Swarm generation parameters
#  MA Cross periods and Trailing stop size
STRATEGY_PARAMS = [
            #OptParam(name, default_value, min_value, max_value, step)
            OptParam('SlowMAPeriod', 20, 10, 30, 2),
            OptParam('FastMAPeriod', 2, 2, 20, 1),
            OptParam('MedianPeriod', 5, 5, 20, 3)
        ]



#
# Swingpoint global filter settings
#

# As a multiplier of volatility
up_factor = 3.0
down_factor = 10.0


## Exo data

In [ ]:
strategyname_global = EXO_File
d, info = matlab.loaddata('../mat/'+strategyname_global+'.mat')

In [ ]:
d.exo.plot()


In [ ]:
info

# Class based strategy

In [ ]:
class StrategyMACrossTrail(StrategyBase):
    def __init__(self):
        # Initialize parent class
        super().__init__()
        
        # Define system's name
        self.name = 'MA Crossing with trailing stop'
        
        # This is a short strategy
        self.direction = -1
        
        # Define optimized params
        self.opts = STRATEGY_PARAMS
        
        self.initialize()
        
    def initialize(self):
        #
        #  Loading EXO quotes from .mat file
        #
        strategyname = strategyname_global
        self.data, info = matlab.loaddata('../mat/'+strategyname+'.mat')
        
        #
        # Estimating transaction costs in base points of price
        #
        
        # No costs
        self.costs = pd.Series(0, self.data.index)
        
        # Flat costs / 1 point of EXO price per side / 2 roundtrip
        #self.costs = pd.Series(12.0, self.data.index)
        
        # Dynamic costs (we could utilize dynamic costs models)
        #  Like slippage calculation on bid/ask data / etc
        # Some meta code (just in my imagination)
        # costmanager = CostManager('EXO.Ticker').LoadSlippageFromDB('2000-01-01', now)
        # self.costs = costmanager.getslippage() + pd.Series(0.1, self.data.index) # Slippage + commission
    
    @property
    def positionsize(self):
        """
        Returns volatility adjuster positions size for strategy
        :return:
        """
            
        # Defining EXO price
        px = self.data.exo
        
        #Test !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        return pd.Series(1.0, index=px.index)

        # Calculate position size adjusted to volatility of EXO
        # Dollar risk per 1 volatility unit
        risk_perunit = 100
        risk_vola_period = 100

        # Calculate volatility unit
        # In this case we use 10-period rolling median of EXO changes
        vola = abs(px.diff()).rolling(risk_vola_period).median()
        # We want to risk 100$ per 1 volatility unit
        #
        # This type of position sizing used for calibration of swarm members
        # After swarm generation and picking we will use portfolio based MM by Van Tharp
        # Tailored for portfolio size and risks of particular client
        return risk_perunit / vola
    
    def calculate(self, params=None):
        #        
        #
        #  Params is a tripple like (50, 10, 15), where:
        #   50 - slow MA period
        #   10 - fast MA period
        #   15 - median period
        #
        #  On every iteration of swarming algorithm, parameter set will be different.
        #  For more information look inside: /notebooks/tmp/Swarming engine research.ipynb
        #       
        
        if params is None:
            # Return default parameters
            period_slow, period_fast, period_median = self.default_opts()
        else:
            # Unpacking optimization params
            #  in order in self.opts definition
            period_slow, period_fast, period_median = params
        
        # Defining EXO price
        px = self.data.exo
        
        #
        #
        # Indicator calculation
        #
        #
        slow_ma = px.rolling(period_slow).mean()
        fast_ma = px.rolling(period_fast).mean()
        # Median based trailing stop
        trailing_stop = px.rolling(period_median).median().shift(1)        
    
        # Enry/exit rules
        entry_rule = CrossDown(fast_ma, slow_ma)
        exit_rule = (CrossUp(fast_ma, slow_ma) ) | (CrossUp(px, trailing_stop))
        
        # Swarm_member_name must be *unique* for every swarm member
        # We use params values for uniqueness 
        swarm_member_name = str((period_slow, period_fast, period_median))        
        
        return swarm_member_name, entry_rule, exit_rule
        

<h1>Backtest results</h2>

In [ ]:
# Initiate the strategy

strategy = StrategyMACrossTrail()

In [ ]:
strategy.positionsize.plot()

In [ ]:
# Running all parameters permutations in swarm
swarm, swarm_stats, swarm_inposition = strategy.run_swarm()

# Analyze the swarm

### Exploriong swarm bounds

In [ ]:
figsize(15,10)
title(strategy.name)
swarm.apply(lambda x: np.percentile(x, q=95), axis=1).plot(label='Top 5%', c='g')
swarm.apply(lambda x: np.percentile(x, q=66), axis=1).plot(label='Top 66%', c='g', style='k--')
swarm.apply(np.median, axis=1).plot(label='Median member')
swarm.apply(lambda x: np.percentile(x, q=33), axis=1).plot(label='Bottom 33%', c='r', style='k--')
swarm.apply(lambda x: np.percentile(x, q=5), axis=1).plot(label='Bottom 5%', c='r')

legend(loc=2);

avg_no_costs = swarm.apply(np.mean, axis=1)

## Picking the swarm members and compose a portfolio


In [ ]:
# Swarm Manager context (settings)
context = {
    'nsystems': NSwarm_Members, # Number of swarm members to pick
}

# Rebalance the swarm every Monday
rebalance_time = swarm.index.dayofweek == 0

# Rank systems by 14 days returns of equity
# FYI: ranking_func must me assigned without () at the end of line!
ranking_func =  SwarmRanker.highestreturns_14days

### Setting up SwarmManager

In [ ]:
swarm_manager = SwarmManager(rebalancetime=rebalance_time,    # Every week
                             rankerfunc=ranking_func,         # Rank - 14 days returns
                             context=context,                 # Backtester settings / params
                            )

In [ ]:
# Getting average swarm member equity
avg_swarm = SwarmManager.get_average_swarm(swarm)

### Swarm global filtering algorithms



In [ ]:
price = avg_swarm
period = 10
vola_offset = 1.0

In [ ]:

def swingpoint_filter_threshold(price, period, up_factor, down_factor):
    #vola = price.diff().abs().rolling(60).median()
    vola = price.diff(period=5).abs().rolling(60).median()
    
    
    swing_point = pd.Series(np.nan, index=price.index)
    swing_point_regime = pd.Series(0, index=price.index)
    
    # Swing point bullish regime
    swing_switch = 1
    
    # Swing point start index
    sw_i = -1
    
    # Min/Max prices for swings
    sw_h_max = price[0]
    sw_l_min = price[0]
    
    
    for i in range(len(price)):
        if i == 0:
            continue
        if np.isnan(price[i]):
            continue
        if np.isnan(vola.values[i]):            
            continue
        elif sw_i == -1 and vola.values[i] > 0:     
            sw_h_max = sw_l_min = price[i]
            sw_i = i
        
        
        if swing_switch == 1:
            #
            #  We have a bullish swing 
            #
            sw_h_max = max(sw_h_max, price[i])
            
            
            # Check for reversion
            if price[i] <= sw_h_max-vola[sw_i]*down_factor:
                # Reverse swing
                swing_switch = -1
                sw_l_min = price.values[i]
                sw_h_max = price.values[i]
                swing_point.values[i] = sw_l_min + vola[sw_i]*up_factor
                
                sw_i = i
            else:
                swing_point.values[i] = sw_h_max-vola[sw_i]*down_factor  
                
                
        else:
            #
            #  We have a bearish swing
            #
            sw_l_min = min(sw_l_min, price.values[i])
            
            
            # Check for reversion
            if price.values[i] >= sw_l_min + vola[sw_i]*up_factor:
                # Reverse swing
                swing_switch = 1
                sw_l_min = price.values[i]
                sw_h_max = price.values[i]
                sw_i = i
                swing_point.values[i] = sw_h_max - vola[sw_i]*down_factor                  
            else:
                swing_point.values[i] = sw_l_min + vola[sw_i]*up_factor  
                
        swing_point_regime.values[i] = swing_switch                
    return swing_point, swing_point_regime
            
            
swing_point_threshold, swing_point_regime_threshold = swingpoint_filter_threshold(price, period, up_factor, down_factor)

price.plot()
swing_point_threshold.plot()
figure()

swing_point_regime_threshold.plot.area(stacked=False)    
    

### Applying global swingpoint factor

In [ ]:
# Simple swing point filter
#global_filter = swing_point_regime == -1


# Swing point filter with volatility threshold
global_filter = swing_point_regime_threshold == -1



In [ ]:
is_picked_df_no_filter = swarm_manager.backtest(swarm)
is_picked_df_with_filter = swarm_manager.backtest(swarm, global_filter)

# Estimating swarm rebalance and global_filter costs

In [ ]:
swarm_picked_inposition_wfilter  = swarm_inposition & is_picked_df_with_filter
swarm_picked_inposition_wofilter  = swarm_inposition & is_picked_df_no_filter


## Transaction count for swarm (in + outs) * swarm_members

In [ ]:
swarm_picked_inposition_wfilter.sum().sum()

In [ ]:
swarm_picked_inposition_wofilter.sum().sum()

### Calculating costs for swarm

In [ ]:
def calc_costs_per_side(exo_info, options_costs, futures_costs):
    """
    Calculates costs per one EXO unit
    """
    pcf = exo_info['pcf']
    pcfqty = exo_info['pcfqty']
    
    costs = 0.0
    
    for i, c in enumerate(pcf):
        if c == 2: # Futures contract
            costs += abs(pcfqty[i] * futures_costs)
        elif c == 0: # Call contract
            costs += abs(pcfqty[i] * options_costs)
        elif c == 1: # Put contract
            costs += abs(pcfqty[i] * options_costs)
        else:
            #Unexpected error
            raise ValueError("Unexpected contact type: {0}".format(c))        
        
    return costs

In [ ]:
swarm_transactions_wfilter = (swarm_picked_inposition_wfilter != swarm_picked_inposition_wfilter.shift()).astype(np.int8)
swarm_entries_wfilter = ( (swarm_picked_inposition_wfilter == 1) & (swarm_picked_inposition_wfilter.shift() == 0))

swarm_transactions_wofilter = (swarm_picked_inposition_wofilter != swarm_picked_inposition_wofilter.shift()).astype(np.int8)
swarm_entries_wofilter = ( (swarm_picked_inposition_wofilter == 1) & (swarm_picked_inposition_wofilter.shift() == 0))


costs = calc_costs_per_side(info, costs_options, costs_futures)

print('Estimated costs ${0}'.format(costs))
info

### Prepearing swarm data with rebalancing costs

In [ ]:
eq_changes_wfilter = swarm.diff()
eq_changes_wofilter = swarm.diff()



# Fixin costs and 1st bar bug
eq_changes_wfilter[swarm_entries_wfilter == 1] = 0
eq_changes_wofilter[swarm_entries_wofilter == 1] = 0




# Applying costs
eq_changes_wfilter[swarm_transactions_wfilter == 1] = -costs
eq_changes_wofilter[swarm_transactions_wofilter == 1] = -costs


figsize(20, 20)


avg_picked_equity_no_filter = eq_changes_wofilter[is_picked_df_no_filter == 1.0].mean(axis=1).cumsum().ffill()
avg_picked_equity_with_filter = eq_changes_wfilter[is_picked_df_with_filter == 1.0].mean(axis=1).cumsum().ffill()

#avg_all_equity = avg_swarm
avg_all_equity = swarm.diff().mean(axis=1).cumsum().ffill()



avg_all_equity.plot(label='Avg Swarm equity');
avg_picked_equity_no_filter.plot(label='Picked Swarm equity NO Filter')
avg_picked_equity_with_filter.plot(label='Picked Swarm equity WITH Filter')
#avg_swarm.rolling(100).mean().plot(label='Avg Swarm MovingAvg filter')
#swing_point.plot(label='SwingPoint')
title('Average swarm member equity');
legend(loc=2)

## Margin graphs

### Margin without global filter

In [ ]:
figsize(10, 10)
mrg = (swarm_picked_inposition_wofilter * info['margin']).sum(axis=1)

mrg.rolling(20).mean().plot(label="Mean Margin")
mrg.rolling(20).max().plot(label="Mean Margin")

### Margin with global filter

In [ ]:
figsize(10, 10)
mrg = (swarm_picked_inposition_wfilter * info['margin']).sum(axis=1)

mrg.rolling(20).mean().plot(label="Mean Margin")
mrg.rolling(20).max().plot(label="Mean Margin")